# Cosas por agregar
- visualización
- filtrado
- incluir en loc y iloc ejemplos temporales y booleanos (ej dfl.loc['20130102':'20130104'])

# Import importantes y cargo pickles de dfs

In [ ]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import pickle


# Download latest version
kagglehub.dataset_download("isathyam31/adult-income-prediction-classification")


In [2]:
#Cargo pickles
with open("df.pkl","rb") as file:
    df=pickle.load(file)
    
with open("df_t.pkl","rb") as file:
    df_t=pickle.load(file)

# Gathering data

## Datos en general

La función más usada, lejos es `read_csv` de pandas, donde basta que demos el path del archivo y su nombre, y se leerá si no hay mayor problema. Por default, la primera fila se transforma en las columnas del dataframe, que podemos ver usando `dataframe.columns`

In [ ]:
path="/home/capitanespiral/Documents/GitHub/data_science" #Aquí introducir tu path
filename="data.csv"

df=pd.read_csv(f"{path}/{filename}")
#Display entrega mucho mejor resultado que print para los dataframes!
display(df)
print(df.columns)

---
Opciones más usadas de `read_csv`:
- **sep o delimiter** &rarr; El separador de la data, usualmente se encuentra solo pero a veces es mejor darlo explícitamente. (en el caso que se confunda el identificador automático)
- **header** &rarr; Número de fila que se toma como nombre de columnas y desde el cual comienza la data, por default, cero (inicio del .csv).
- **skiprows** &rarr; Filas a saltar desde el inicio del archivo (indexeando desde cero, se puede entregar lista o tupla)
- **skipfooter** &rarr; Filas a saltar desde el final del archivo.
- **usecols** &rarr; Subset de columnas a usar (secuencia de números o nombres explícitos).

In [ ]:
#Si la primera fila está mala, cambiamos el header
df1=pd.read_csv("data_bad_first_row.csv") #Salió pésimo
df2=pd.read_csv("data_bad_first_row.csv",header=1) #Mucho mejor

display(df1)
display(df2)

In [ ]:
#Si la segunda está mala
df3=pd.read_csv("data_bad_second_row.csv") #Primer dato pésimo
df4=pd.read_csv("data_bad_second_row.csv",skiprows=[1],header=0) #Nos saltamos la segunda fila y conservamos el header

display(df3)
display(df4)

In [ ]:

#Seleccionemos ciertas columnas no más
df5=pd.read_csv("data_bad_first_row.csv",header=1,usecols=[0,3]) #Puede ser con números, aquí la primera y la cuarta
df6=pd.read_csv("data_bad_first_row.csv",header=1,usecols=range(4)) #Puede ser con iteradores, acá me entrega de la primera A la cuarta
df7=pd.read_csv("data_bad_first_row.csv",header=1,usecols=["age","education","workclass"]) #Puede ser con los nombres explícitos

display(df5)
display(df6)
display(df7)

In [ ]:
del df1,df2,df3,df4,df5,df6,df7

## Series de Tiempo

Para el caso de series de tiempo, hay que saber trabajar con `Timestamps`, `Datetime`, `Timedelta`, etc...

### Funciones generales 

- **pd.to_datetime()** &rarr; Transforma lo que le entregues en un "Datetime", funciona aceptando varios formatos y si le entregas secuencias.
- **pd.date_range()** &rarr; Para crear puntos en el tiempo equiespaceados, con un "start" un "end", posibilidad de "periods" o "freq"
    - **freq** más comunes:"y" (años),"m" (meses),"W" (semana), "B" (business day), "d" (días), "h" (horas), "min" (minutos), "s" (segundos) Ojo que hay *infinitas* opciones! revisar en https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
- **df.resample().func()** &rarr; Entregando como variable una frecuencia, podemos "resamplear" según la función "func"
- **datetime.min() y .max()** &rarr; Evidente
- **datetimeindex.atributos** &rarr; nos permite acceder a muchas funciones de tiempo como "hour","day","minute","second","dayofyear", "strftime", entre otras (si es una serie, no un índice, agregar dt) 

Ejemplos de estas funciones:

In [ ]:
##datetime con varios formatos
dt1=pd.to_datetime(["13/1/2018", np.datetime64("2018-01-13"), datetime.datetime(2018, 1, 13)],dayfirst=True)
print("dt1:",dt1,"\n")

##date_range
#Creando una lista de tiempo cada tres horas
t1="1-1-2000"
t2="2000-3-10"
dt2=pd.date_range(start=t1,end=t2,freq="3h")
print("dt2:",dt2,"\n")

#Lo mismo pero con 15 minutos
t1="1-1-2000"
t2="2000-3-10"
dt3=pd.date_range(start=t1,end=t2,freq="15min")
print("dt3:",dt3,"\n")

#Si tengo periodos y frecuencia
dt4=pd.date_range(start="2018-8-1", periods=5, freq="2d")
print("dt4:",dt4,"\n")


In [ ]:
#Como usar el resampleo
idx = pd.date_range("2018-01-01", periods=10, freq="h") #ojo que 1h = h
ts = pd.Series(range(len(idx)), index=idx)
print("ts",ts,"\n",sep="\n")
#Downsample (también muy usado el sum)
ts_downsampled=ts.resample("2h").mean() #LEJOS el más útil
print("ts_downsampled",ts_downsampled,"\n",sep="\n")
#upsample (también muy usado bfill)
ts_upsampled=ts.resample("30min").ffill() #Sería interesante, acá interpolar
print("ts_upsampled",ts_upsampled,"\n",sep="\n")

In [16]:
del dt1,dt2,dt3,dt4,idx,ts,ts_upsampled,ts_downsampled

Ahora trabajemos con data de verdad:

In [ ]:
df_t=pd.read_csv("biomet1.csv",skiprows=[1],header=0) #Tenemos caso de segunda linea sin sentido
df_t["time_t"]=pd.to_datetime(df_t['date']+" "+df_t['time']) #Lo guardamos en una nueva columna
display(df_t)
#Podemos acceder a propiedades o métodos específicos usando el "dt"
print("Solo el año:",df_t["time_t"].dt.year,"\n",sep="\n")
print("Cambio el formato:",df_t["time_t"].dt.strftime("%Y/%m/%d"),"\n",sep="\n")

In [ ]:
#Fijamos un nuevo índice
df_t=df_t.set_index("time_t") #Esto elimina la columna "time_t"
#Al fijarlo como índice, ahora podemos accesar simplemente con ".atributo"
display(df_t)
print(df_t.index.day)
print(df_t.index.dayofyear)

In [ ]:
#Ahora cubrimos todo el espacio
ti=df_t.index.min()
tf=df_t.index.max()
time_total=pd.date_range(start=ti,end=tf,freq="30min")
df_t=df_t.reindex(time_total) #Esta función redefine el índice, conservando datos y agregando nans si el nuevo índice coincide con el anterior o si no existe (para cada fila!)
display(df_t)

## Almacenamos como pickle los dataframes

In [7]:
#El primer dataframe
with open('df.pkl',"wb") as file:
    pickle.dump(df,file)

#El dataframe de serie de tiempo
with open('df_t.pkl',"wb") as file:
    pickle.dump(df_t,file)

# See data and basic statistics

## Indexar

Primero entender que *pandas* tiene dos grandes tipos de objetos: `Series`y `DataFrame`. El primero es como un "vector" (unidimensional) y el segundo está compuesto por varias `Series`, siendo como una "matriz". Y cada uno siempre tiene asociado su `Index`, que puede ser numérico o de cualquier naturaleza (uno común, temporal).

La forma básica de indexar cada tipo tiene distintos resultados:
- **Series[label]** &rarr; Dará un valor escalar en tal posición acorde al `Index`.
- **DataFrame[colname]** &rarr; Dará una *Serie* correspondiente a tal nombre de columna

In [ ]:
#Accedo a una columna, me entrega a una serie, y despues a un elemento en específico
print("**Dataframe**")
print("Edad",df["age"],"\n",sep="\n")
print("Tercer valor:",df["age"][2],"\n")

#Si el índice no es numérico, esto igual se puede hacer, pero se recomienda indexar acorde al índice
print("**Dataframe temporal**")
print("Calor sensible",df_t["H"],"\n",sep="\n") #Notar que el índice es temporal
print("Tercer valor indexando con 'int':",df_t["H"][2]) #Obtengo el tercer valor -> Notar advertencia de pandas!
print("Tercer valor indexando con un 'label' del 'index':",df_t["H"]["2014-01-01 01:30:00"]) #Lo mismo pero con el tiempo, engorroso pero preciso

Siempre podemos acceder a **varios valores** de una *serie* (con indexación típica de python) o **varias columnas** en un *dataframe* (agregando un nuevo []:). Notar que esto último entrega un nuevo `DataFrame` que no permite indexación "típica" de python.

In [ ]:
display(df[["age","education"]])
#display(df[["age","education"]][1:5]) #Esto tira error
print("Del segundo al quinto valor de una columna:",df["age"][1:5],sep="\n")

También usando slices de enteros en una *serie* o *dataframe* (en este caso se accesarán filas, no columnas)

In [ ]:
#Indexado clásico de python sobre una serie
print("Del primer al quinto dato:",df["age"][:5],"\n",sep="\n") 
print("Los datos pares:",df["age"][::2],"\n",sep="\n")

#Indexado clásico de python sobre un dataframe
print("\nDel primer al quinto dato:")
display(df_t[:5]) 
print("\nLos datos dados vuelta:")
display(df_t[::-1])

Para indexar también se tienen los métodos `.loc` y `.iloc`(para *series* y *dataframes*)
1. `.loc` &rarr; trabaja con labels del índice o con arreglos booleanos.
2. `.iloc` &rarr; trabaja con posisiones con enteros (desde 0 a -1) o con arreglos booleanos.

In [ ]:
df_temp=df.drop([1]) #Boto la segunda fila
display(df_temp)

print("Con loc",df_temp.loc[2],"\n",sep="\n") #Aquí 2 es interpretado como "label", será la segunda fila

print("Con loc",df_temp.iloc[2],"\n",sep="\n") #Aquí como entero, será la tercera fila

In [ ]:
#Usando listas de elementos
print("Usando listas:")

display(df_temp.loc[[0,3,5]]) #El label 0, 3, 5

display(df_temp.iloc[[0,3,5]]) #Las posiciones asociados (otro label)

#También usando "slices"
print("\nUsando slices ahora:")

display(df_temp.loc[0:5])

display(df_temp.iloc[0:5])

print("En este caso funcionan igual por la naturaleza de este índice. NOTAR que se incluye el PRINCIPIO Y EL FINAL!")

In [ ]:
#Mezclando filas y columnas loc
print("Slice y columna - loc",df_temp.loc[2:9,"age"],"\n",sep="\n")

print("Slice y slice - loc")
display(df_temp.loc[2:9,"age":"sex"])

print("\nFilas y columnas - loc")
display(df_temp.loc[[3,5,7],["age","country","race"]])

In [ ]:
#Mezclando filas y columnas iloc

print("Slice y columna - iloc",df_temp.iloc[2:9,0],"\n",sep="\n")

print("Slice y slice - iloc")
display(df_temp.iloc[2:9,0:7])

print("\nFilas y columnas - iloc")
display(df_temp.iloc[[3,5,7],[0,5,7]])

Hay una serie de funciones que nos permiten visualizar la data. Dentro de las más básicas:
- **df.head(n)** &rarr; nos permite ver las primeras "n" filas (sin n, por default 5), funciona para `Series` y `DataFrame`.
- **df.tail(n)** &rarr; nos permite ver las últimas "n" filas (lo mismo), funciona para `Series` y `DataFrame`.

In [ ]:
#Para dataframes
display(df.head())
display(df.head(10))

#Para una serie
print(df["age"].head())

In [ ]:
display(df.tail())

display(df["workclass"].tail())

# Links de interés
- Datasets https://www.kaggle.com/datasets
- Documentación de Pandas https://pandas.pydata.org/docs/index.html 
- Cookbook de pandas https://pandas.pydata.org/docs/user_guide/cookbook.html
- Estructura y un par de cosas sobre data engineering https://apmonitor.com/pds/index.php/Main/DataPreparation